In [16]:

import numpy as np
import scipy.sparse.linalg as lg
import time
from copy import deepcopy

# PQR

In [17]:
def PQR(matrix, n, m, r=0, eps=0):
    max_rank = min(n, m)
    if r <= 0 or r > max_rank:
        r = max_rank

    Q = np.zeros((n, r), dtype=matrix.dtype)
    R = np.zeros((r, m), dtype=matrix.dtype)
    perm = list(range(m))

    k = 0
    resid = eps + 1.0
    A = matrix 

    while k < r and resid > eps:
        sqs = [(A[:, j] @ A[:, j]) for j in range(k, m)]
        resid = np.sqrt(sum(sqs))

        rel = max(range(len(sqs)), key=lambda idx: sqs[idx])
        pivot = k + rel

        perm[k], perm[pivot] = perm[pivot], perm[k]
        A[:, [k, pivot]] = A[:, [pivot, k]]
        R[:, [k, pivot]] = R[:, [pivot, k]]

        diag = np.sqrt(sqs[rel])
        R[k, k] = diag
        u = A[:, k] / diag
        Q[:, k] = u

        for j in range(k + 1, m):
            c = float(u @ A[:, j])
            R[k, j] = c
            A[:, j] -= c * u

        k += 1

    return Q, R, np.array(perm), k


# MAXVOL

In [18]:
def maxvol(A, r=None, eps=1e-3, it_max=10_000):
    n, m = A.shape
    if r is None or r > min(n, m):
        r = min(n, m)

    row_perm = np.random.permutation(n)
    init_rows = row_perm[:r]

    _, _, col_perm, _ = PQR(A[init_rows, :], r, m, r)

    rows = init_rows.copy()
    cols = col_perm[:r].copy()

    iters = 0
    while iters < it_max:
        iters += 1

        submat = A[np.ix_(rows, cols)]              
        inv_sub = np.linalg.inv(submat)
        C = A[:, cols] @ inv_sub               

        i, j = divmod(np.abs(C).argmax(), r)
        max_val = abs(C[i, j])

        if max_val <= 1 + eps:                       
            break

        if i not in rows:
            rows[j] = i
            continue
        C_cols = inv_sub @ A[rows, :]
        p, q = divmod(np.abs(C_cols).argmax(), m)
        if abs(C_cols[p, q]) > 1 + eps and q not in cols:
            cols[p] = q
        else:
            break

    return rows, cols, iters


# Cross Approx

In [19]:
def cross_approx(A, r=None, eps=1e-12):
    n, m = A.shape
    if r is None or r > min(n, m):
        r = min(n, m)

    Q = np.zeros((n, r), dtype=A.dtype)
    R = np.zeros((r, m), dtype=A.dtype)

    free_cols = np.arange(m)         
    k = 0                           
    max_el = eps + 1                 


    while k < r and max_el * np.sqrt((n - k) * (m - k)) > eps:


        j = np.random.randint(free_cols.size)
        col = free_cols[j]

        residual_col = A[:, col] - Q[:, :k] @ R[:k, col]
        row = np.abs(residual_col).argmax()


        residual_row = A[row, :] - Q[row, :k] @ R[:k, :]
        col = np.abs(residual_row).argmax()


        residual_col = A[:, col] - Q[:, :k] @ R[:k, col]
        row = np.abs(residual_col).argmax()
        residual_row = A[row, :] - Q[row, :k] @ R[:k, :]

        pivot = residual_col[row]
        if pivot == 0:
            break                     

        Q[:, k] = residual_col / pivot
        R[k, :] = residual_row

        max_el = abs(pivot)

        free_cols = np.delete(free_cols, np.where(free_cols == col))
        k += 1

    return Q[:, :k], R[:k, :], k


# R-SVD

In [20]:
def rsvd_simple(A, r, oversample=None):

    n, m = A.shape
    if r > min(n, m):
        raise ValueError("r must not exceed min(n, m)")

    if oversample is None:
        oversample = max(1, int(np.ceil(np.log10(min(n, m)) * r)))
    p = r + oversample               


    transposed = False
    if n > m:
        A = A.T
        n, m = m, n
        transposed = True

    N = np.random.randn(m, p)


    Y = A @ N

    Q, _ = np.linalg.qr(Y, mode="reduced")


    B = Q.T @ A                          
    U_hat, s, Vt = lg.svds(B, k=r)       

    U = Q @ U_hat                      
    V = Vt.T                            


    if transposed:
        return V, s[::-1], U
    else:
        return U[:, ::-1], s[::-1], V[:, ::-1]


# Testing

In [21]:
n = 1000
m = 1000
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = 1 / (i + j + 1)


matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# ---------------------- Run PQR ----------------------
print("PQR")
start_time = time.time()

Q, R, perm, rank = PQR(matrix, n, m, r=30, eps=1e-10)

print(f"time = {time.time() - start_time}")

rel_err = np.linalg.norm(Q @ R - matrix[:, perm]) / np.linalg.norm(matrix)
print(rel_err)
print(f"rank = {rank}")

# --------------------- Run MAXVOL --------------------
print("MAXVOL")
start_time = time.time()

rows, cols, iters = maxvol(matrix, r=30, eps=1e-3)

print(f"time = {time.time() - start_time}")

submat  = matrix[np.ix_(rows, cols)]         # 30 × 30
approx  = matrix[:, cols] @ np.linalg.inv(submat) @ matrix[rows, :]

rel_err = np.linalg.norm(approx - matrix) / np.linalg.norm(matrix)
print(rel_err)
print(f"iters = {iters}")


# ---------------------Cross Approx --------------------
print("CROSS APPROX")
start_time = time.time()

Q, R, rank = cross_approx(matrix, r=30, eps=1e-10)

print(f"time = {time.time() - start_time}")

rel_err = np.linalg.norm(Q @ R - matrix) / np.linalg.norm(matrix)
print(rel_err)
print(f"rank = {rank}")

# -------------------- Run R-SVD --------------------
print("R-SVD")
start_time = time.time()

U, s, V = rsvd_simple(matrix, r=24)        

print(f"time = {time.time() - start_time}")

rel_err = np.linalg.norm((U * s) @ V.T - matrix) / np.linalg.norm(matrix)
print(rel_err)


PQR
time = 0.26044344902038574
1.8943000033400645
rank = 22
MAXVOL
time = 0.26175355911254883
0.00031548966388146484
iters = 32
CROSS APPROX
time = 0.004407167434692383
1.5898787038504245e-12
rank = 23
R-SVD
time = 0.037979841232299805
8.783772744293547e-06
